# 04 - The Hydrogen Molecular Ion $H_2^+$

**Overview** 

This notebook guides you through ...  

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from scipy.special import genlaguerre, sph_harm, factorial
from numpy.polynomial.legendre import leggauss
from numpy.polynomial.laguerre import laggauss
from scipy.linalg import eigh
from functools import partial

In [ ]:
# @title Utility Functions  { display-mode: "form" }
# -------------------- Basis Set Generation --------------------
def sph_coords(rvec):
    """
    Convert Cartesian coordinates to spherical coordinates.
    """
    x, y, z = rvec[...,0], rvec[...,1], rvec[...,2]
    r = np.sqrt(x*x + y*y + z*z)
    # theta: handle r=0 safely
    theta = np.where(r > 0, np.arccos(np.clip(z / (r + 1e-300), -1.0, 1.0)), 0.0)
    phi = np.arctan2(y, x)
    return r, theta, phi

def R_nl(n, l, r, Z=1):
    """
    Hydrogenic radial wavefunction (atomic units, a0=1).
    Returns R_{n,l}(r).
    """
    # Normalization constant
    prefac = np.sqrt((2.0*Z/n)**3 * factorial(n-l-1) / (2*n*factorial(n+l)))
    rho = 2.0 * Z * r / n
    L = genlaguerre(n-l-1, 2*l+1)(rho)  # Associated Laguerre polynomial

    return prefac * np.exp(-rho/2) * rho**l * L

def hydrogen_orbital(n, l, m, r0, rvec, Z=1):
    """
    Full hydrogenic orbital psi_{n,l,m}(rvec-r0).
    - rvec: cartesian coordinate (array)
    - r0: center of orbital (array)
    """
    r, theta, phi = sph_coords(rvec - r0)
    R = R_nl(n, l, r, Z)
    Y = sph_harm(m, l, phi, theta)  # scipy uses (m,l,phi,theta)
    return R * Y

def hydrogen_energy(n, Z=1):
    """
    Energy of hydrogen-like orbital with principal quantum number n,
    nuclear charge Z (default hydrogen), in Hartree atomic units.

    E_n = - Z^2 / (2 n^2)
    """
    if n < 1 or int(n) != n:
        raise ValueError("n must be a positive integer")
    return - (Z**2) / (2.0 * n**2)

# -------------------- Partition of unity weights --------------------
def weights_RA_RB(r, RA, RB, alpha=3.0):
    dA = np.linalg.norm(r - RA)
    dB = np.linalg.norm(r - RB)
    R  = np.linalg.norm(RB - RA)
    s  = np.tanh(alpha * (dB - dA) / (R + 1e-300))
    return 0.5*(1 - s), 0.5*(1 + s)  # wA, wB

# -------------------- Atom-centered spherical quadrature --------------------
def integrate_atom_centered(f, R0, n_rad=40, n_th=20, n_ph=24, alpha_rad=1.0):
    """
    ∫ f(r) d^3r using spherical coords about R0.
    Radial: Gauss–Laguerre with mapping r = alpha_rad * x (weight e^{-x})
    Angular: Gauss–Legendre in cosθ; φ uniform
    """
    # Radial Gauss–Laguerre for ∫_0^∞ e^{-x} g(x) dx
    x_r, w_r = laggauss(n_rad)
    r = alpha_rad * x_r
    # Angular nodes/weights
    u, w_th = leggauss(n_th)           # u = cosθ in [-1,1]
    th = np.arccos(u)
    ph = np.linspace(0.0, 2.0*np.pi, n_ph, endpoint=False)
    w_ph = (2.0*np.pi)/n_ph * np.ones_like(ph)

    total = 0.0
    for ri, wri in zip(r, w_r):
        # Jacobian: (alpha^3) * r^2 e^{-r/alpha} * sinθ
        pref_r = alpha_rad * wri * np.exp(ri/alpha_rad) * (ri**2)
        sin_th = np.sin(th)
        for thi, wti, sin_i in zip(th, w_th, sin_th):
            for phi, wpi in zip(ph, w_ph):
                x_i = R0[0] + ri*np.sin(thi)*np.cos(phi)
                y_i = R0[1] + ri*np.sin(thi)*np.sin(phi)
                z_i = R0[2] + ri*np.cos(thi)
                jac = pref_r * wti * wpi
                total += f(np.array([x_i, y_i, z_i])) * jac
    return total

# -------------------- Overlap S integral for orbitals i,j ----------------------------------
def S_i_j(orbital_i, orbital_j, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):
    """
    S_i_j = ∫ φ_{j}(r)* x φ_{i}(r) d^3r
    Evaluated with two-grid partition-of-unity by default.
    """

    def raw_integrand(r):
        val = orbital_i(rvec = r)*np.conj(orbital_j(rvec = r))
        return val

    # Two-grid split with smooth weights
    def fA(r):
        wA, _ = weights_RA_RB(r, RA, RB)
        return wA * raw_integrand(r)

    def fB(r):
        _, wB = weights_RA_RB(r, RA, RB)
        return wB * raw_integrand(r)

    IA = integrate_atom_centered(fA, RA, n_rad, n_th, n_ph, alpha_rad=alpha)
    IB = integrate_atom_centered(fB, RB, n_rad, n_th, n_ph, alpha_rad=alpha)
    return IA + IB

# ----------- Coulomb J integral for orbitals i and j on center opposite to i --------------
def J_i_j(orbital_i, orbital_j, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):
    """
    J_iA^(B)(R) = -∫ φ_{j}(r)* x φ_{i}(r) / |r - RB| d^3r
    Evaluated with two-grid partition-of-unity by default.
    """

    def raw_integrand(r):
        # Compute Coulomb integral with nucleus opposite to i-th orbital center
        R0 = -orbital_i.keywords.get("r0")
        val = orbital_i(rvec = r)*np.conj(orbital_j(rvec = r))
        return - val / (np.linalg.norm(r - R0) + 1e-300)

    # Two-grid split with smooth weights
    def fA(r):
        wA, _ = weights_RA_RB(r, RA, RB)
        return wA * raw_integrand(r)

    def fB(r):
        _, wB = weights_RA_RB(r, RA, RB)
        return wB * raw_integrand(r)

    IA = integrate_atom_centered(fA, RA, n_rad, n_th, n_ph, alpha_rad=alpha)
    IB = integrate_atom_centered(fB, RB, n_rad, n_th, n_ph, alpha_rad=alpha)
    return IA + IB

# Build Overlap Matrix
def overlap_matrix(basis, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):

    S = np.zeros((len(basis),len(basis)), dtype=np.complex128)

    for i, ket in enumerate(basis):
        for j, bra in enumerate(basis):
            if i == j:
                # the basis is normalized
                S[i,j] = 1.0
            elif i < j :
                # if the two basis have the same center, they should be orthogonal
                if np.array_equal(ket.keywords.get("r0"), bra.keywords.get("r0")):
                    S[i,j] = 0.0
                else:
                    # otherwise, compute the overlap
                    S[i,j] = S_i_j(ket, bra, RA, RB, n_rad=20, n_th=20, n_ph=12, alpha=1.0)
                    S[j,i] = np.conj(S[i,j])
    return S

# Build Hamiltonian Matrix
def hamiltonian_matrix(basis_vectors, basis_energies, RA, RB, n_rad=40, n_th=20, n_ph=24, alpha=1.0):

    H = np.zeros((len(basis_vectors),len(basis_vectors)), dtype=np.complex128)
    S = overlap_matrix(basis_vectors, RA, RB, n_rad, n_th, n_ph, alpha)

    # we can exploit symmetry to compute only the two top halves of the top quarters of the matrix
    for i, (ket, ei) in enumerate(zip(basis_vectors[::2],basis_energies[::2])):
        i_index = i*2 # actual index in the full basis
        for j, bra in enumerate(basis_vectors[i_index::2]):
            j_index = i_index + 2*j
            H[i_index,j_index] = ei*S[i_index,j_index] + J_i_j(ket, bra, RA, RB, n_rad, n_th, n_ph, alpha)
            H[j_index,i_index] = np.conj(H[i_index,j_index])
            H[i_index+1,j_index+1] = H[i_index,j_index]
            H[j_index+1,i_index+1] = np.conj(H[i_index,j_index])
        for j, bra in enumerate(basis_vectors[i_index+1::2]):
            j_index = i_index + 1 + 2*j
            H[i_index,j_index] = ei*S[i_index,j_index] + J_i_j(ket, bra, RA, RB, n_rad, n_th, n_ph, alpha)
            H[j_index,i_index] = np.conj(H[i_index,j_index])
            H[i_index+1,j_index-1] = H[i_index,j_index]
            H[j_index-1,i_index+1] = np.conj(H[i_index,j_index])
    return H, S

# -------------------- Generalized Eigenvalue Problem Solver --------------------
def generalized_eigh_lowdin(H, S, rel_cut=5e-2):
    # S = U diag(w) U^H
    w, U = eigh(S)
    thr = rel_cut * w.max()
    keep = w > thr
    if not np.any(keep):
        raise np.linalg.LinAlgError("All overlap eigenvalues below threshold.")
    Uk, wk = U[:, keep], w[keep]

    S_inv_sqrt = (Uk / np.sqrt(wk)) @ Uk.conj().T
    H_tilde    = S_inv_sqrt @ H @ S_inv_sqrt
    E, Ctil    = eigh(H_tilde)
    C          = S_inv_sqrt @ Ctil           # AO-space eigenvectors; C^H S C ≈ I
    return E, C, keep, w

def ghost_score(S, c, lam_cut=1e-2, relative=True):
    # S = U diag(λ) U^H
    lam, U = eigh(S)
    if relative:
        lam_cut = lam_cut * lam.max()
    y = U.conj().T @ c
    small = lam < lam_cut
    # fraction of Euclidean weight living in small-λ subspace
    num = np.sum(np.abs(y[small])**2)
    den = np.sum(np.abs(y)**2)
    return float(num/den), lam, U
# Example usage:
#scores = [ghost_score(S, C[:,k])[0] for k in range(C.shape[1])]
#print(scores)
# A ghost will have score ~ 1.0

**Problem** 

Lorem Ipsum ... 

**Model**

Lorem Ipsum ...

>Smart question?

Lorem Ipsum ...

# Hydrogenic Radial Functions and Spherical Harmonics

## Radial Functions $R_{n\ell}(r)$ (normalized)
For a hydrogen-like ion with nuclear charge $Z$ and Bohr radius $a_0$,
define the dimensionless variable
$$
\rho \equiv \frac{2 Z r}{n a_0}.
$$
The normalized radial functions are
$$
R_{n\ell}(r)
= \left(\frac{2Z}{n a_0}\right)^{\!\!3/2}
\sqrt{\frac{(n-\ell-1)!}{2n\, (n+\ell)!}}\;
e^{-\rho/2}\,\rho^{\ell}\,
L_{n-\ell-1}^{(2\ell+1)}(\rho),
$$
where $L_{k}^{(\alpha)}$ are the associated Laguerre polynomials. They obey
$$
\int_{0}^{\infty} |R_{n\ell}(r)|^2\, r^2\,dr = 1.
$$

### Examples (with $Z=1$)
$$
R_{10}(r) = 2\,a_0^{-3/2}\,e^{-r/a_0},\qquad
R_{21}(r) = \frac{1}{2\sqrt{6}}\,a_0^{-3/2}\,\frac{r}{a_0}\,e^{-r/(2a_0)}.
$$

## Spherical Harmonics $Y_{\ell}^{m}(\theta,\phi)$ (normalized)
With the Condon–Shortley phase convention,
$$
Y_{\ell}^{m}(\theta,\phi)
= (-1)^m \sqrt{\frac{2\ell+1}{4\pi}\,\frac{(\ell-m)!}{(\ell+m)!}}\;
P_{\ell}^{m}(\cos\theta)\, e^{i m \phi},
\quad \ell=0,1,2,\dots,\; m=-\ell,\dots,\ell,
$$
where $P_{\ell}^{m}$ are the associated Legendre functions. They satisfy
$$
\int_{0}^{2\pi}\!\!\int_{0}^{\pi}
Y_{\ell}^{m}(\theta,\phi)\,Y_{\ell'}^{m'}(\theta,\phi)^{*}\,
\sin\theta\, d\theta\, d\phi
= \delta_{\ell\ell'}\,\delta_{mm'}.
$$

### Low-order Examples
$$
Y_{0}^{0} = \frac{1}{\sqrt{4\pi}},\qquad
Y_{1}^{0} = \sqrt{\frac{3}{4\pi}}\,\cos\theta,\qquad
Y_{1}^{\pm1} = \mp \sqrt{\frac{3}{8\pi}}\,\sin\theta\, e^{\pm i\phi}.
$$

## Full Hydrogenic Orbitals
The normalized hydrogenic eigenfunctions are
$$
\psi_{n\ell m}(r,\theta,\phi) \;=\; R_{n\ell}(r)\, Y_{\ell}^{m}(\theta,\phi),
$$
with the orthonormality
$$
\int \psi_{n\ell m}(\mathbf r)\, \psi_{n'\ell' m'}(\mathbf r)^{*}\, d^3 r
= \delta_{nn'}\,\delta_{\ell\ell'}\,\delta_{mm'}.
$$


**Questions**

Before you run any simulation, answer the following question(s):

1. Something
2. Seomthing else

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Other questions

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
distance = 1.0 # @param {type:"number"}
nmax = 3  # @param {type:"integer"}
lmax = 1  # @param {type:"integer"}
integration_accuracy = "coarse"  # @param ["coarse", "medium", "fine", "very fine"]
logical = True # @param {type:"boolean"}

In [ ]:
# @title Setup and Solve the Generalized Eigenvalue Problem { display-mode: "form" }

RA = np.array([-distance/2, 0.0, 0.0])  # nucleus A at -R/2 on x-axis
RB = np.array([ distance/2, 0.0, 0.0])  # nucleus B at +R/2 on x-axis

ns = []
orbitals = []
for i in range(1,nmax+1):
    for j in range(0, min(lmax+1,i)):
        for m in range(-j, j+1):
            ns.append(i)
            orbitals.append(partial(hydrogen_orbital, n=i, l=j, m=m))

basis_vectors = []
basis_energies = []
for n, orbital in zip(ns, orbitals):
    for atom_position in [RA, RB]:
        basis_vectors.append(partial(orbital, r0=atom_position))
        basis_energies.append(hydrogen_energy(n))

if integration_accuracy == "coarse":
    n_rad, n_th, n_ph = 10, 10, 12
elif integration_accuracy == "medium":
    n_rad, n_th, n_ph = 20, 20, 24
elif integration_accuracy == "fine":
    n_rad, n_th, n_ph = 40, 30, 24
elif integration_accuracy == "very fine":
    n_rad, n_th, n_ph = 50, 40, 36

H, S = hamiltonian_matrix(basis_vectors, basis_energies, RA, RB, n_rad, n_th, n_ph, alpha=1.0)
E, C = generalized_eigh_lowdin(H, S, rel_cut=5e-2)

In [ ]:
# @title Visualize the Results { display-mode: "form" }


In [ ]:
x = np.linspace(-3,3,100)
y = np.linspace(-3,3,100)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
rvec = np.stack([X, Y, Z], axis=-1)   # shape (100,100,3)
psi = basis_vectors[1](rvec=rvec)
plt.figure()
plt.contourf(X, Y, psi.real, levels=50, cmap='RdBu')
plt.colorbar(label='Re(ψ)')
plt.xlabel('x (bohr)')
plt.ylabel('y (bohr)')
plt.title('Orbital in z=0 plane')
plt.show()

In [ ]:
import matplotlib.colors as mcolors

# Suppose S is your overlap matrix (complex Hermitian)
data = S.real


# Colormap centered at 0 (white)
norm = mcolors.TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1)

fig, ax = plt.subplots()
cax = ax.matshow(data, cmap='seismic', norm=norm)

# Build the colorbar with custom ticks
cbar = plt.colorbar(cax, ax=ax)
cbar.set_label("Overlap value")
# force symmetric ticks
ticks = np.linspace(data.min(), data.max(), 7)  # 7 evenly spaced ticks
cbar.set_ticks(np.linspace(-1,1,9))

# Annotate each cell with its value
for (i, j), val in np.ndenumerate(data):
    ax.text(j, i, f"{val:.2f}", ha='center', va='center', color='white')

ax.set_title("Overlap Matrix (real part)")
plt.show()

In [ ]:
import matplotlib.colors as mcolors

data = H.real

absolute_max = np.max(np.abs(data))
# Colormap centered at 0 (white)
norm = mcolors.TwoSlopeNorm(vmin=-absolute_max, vcenter=0, vmax=absolute_max)

fig, ax = plt.subplots()
cax = ax.matshow(data, cmap='seismic', norm=norm)

# Build the colorbar with custom ticks
cbar = plt.colorbar(cax, ax=ax)
cbar.set_label("Hamiltonian value")
# force symmetric ticks
ticks = np.linspace(data.min(), data.max(), 7)  # 7 evenly spaced ticks
cbar.set_ticks(np.linspace(-1,1,9))

# Annotate each cell with its value
for (i, j), val in np.ndenumerate(data):
    ax.text(j, i, f"{val:.2f}", ha='center', va='center', color='white')

ax.set_title("Hamiltonian Matrix (real part)")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_energy_levels(energies,
                       tol=2e-4,        # Ha: max gap within a "degenerate" group
                       zero_cut=1e-6,   # Ha: drop |E| <= zero_cut (nullspace)
                       title=None,
                       savepath=None,
                       show=True):
    """
    Plot an energy-level diagram with nearly-degenerate levels side-by-side.

    Parameters
    ----------
    energies : array-like
        1D list/array of energies (Hartree).
    tol : float
        Two levels are grouped if consecutive sorted energies differ by <= tol.
    zero_cut : float
        Discard energies with |E| <= zero_cut.
    title : str or None
        Optional plot title.
    savepath : str or None
        If given, save the figure to this path.
    show : bool
        If True, call plt.show() at the end.

    Returns
    -------
    groups : list[list[float]]
        Energies grouped by near-degeneracy (sorted within each group).
    fig, ax : matplotlib Figure and Axes
    """
    E = np.asarray(energies, dtype=float)
    E = np.sort(E[np.abs(E) > float(zero_cut)])
    if E.size == 0:
        raise ValueError("No energies remain after zero_cut filtering.")

    # Group by near-degeneracy
    groups = []
    cur = [E[0]]
    for e in E[1:]:
        if abs(e - cur[-1]) <= tol:
            cur.append(e)
        else:
            groups.append(cur)
            cur = [e]
    groups.append(cur)

    # Plot
    fig, ax = plt.subplots(figsize=(7, 5))
    for i, grp in enumerate(groups):
        n = len(grp)
        # offsets so near-degenerate levels appear side-by-side
        xs = i + (np.linspace(-0.2, 0.2, n) if n > 1 else np.array([0.0]))
        for x, val in zip(xs, grp):
            ax.hlines(val, x - 0.28, x + 0.28)   # small horizontal “tick”
        # optional: label multiplicity above group
        if n > 1:
            ax.text(i, max(grp) + 0.02, f"{n}×", ha="center", va="bottom", fontsize=9)

    ax.set_xlim(-0.6, len(groups) - 0.4)
    ax.set_xlabel("Group (near-degenerate levels)")
    ax.set_ylabel("Energy (Ha)")
    ax.axhline(0.0, linestyle="--", linewidth=0.8)
    if title is None:
        title = f"Energy levels (grouped, |ΔE| ≤ {tol:g} Ha)"
    ax.set_title(title)
    ax.grid(True, axis="y", linestyle=":", linewidth=0.5)

    fig.tight_layout()
    if savepath:
        fig.savefig(savepath, dpi=200, bbox_inches="tight")
    if show:
        plt.show()
    return groups, fig, ax

groups, fig, ax = plot_energy_levels(E, tol=2e-4, zero_cut=1e-6,
                                     title="Energy level diagram (near-degeneracy)")
